In [1]:
pip install pyspark 

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession
import pyspark as ps
spark = (SparkSession
         .builder
         .appName("Python Spark")
         .config('spark.some.config.option', 'some value')
         .getOrCreate())

In [3]:
conf = ps.SparkConf().setMaster("yarn-client").setAppName("sparK-mer")
conf.set("spark.executor.heartbeatInterval","3600s")

In [4]:
spark.version

'3.5.1'

In [5]:
df_spark = (spark.read.format("csv")
      .option("header","true")
      .load("Cancer_Data.csv"))
df_spark

DataFrame[id: string, diagnosis: string, radius_mean: string, texture_mean: string, perimeter_mean: string, area_mean: string, smoothness_mean: string, compactness_mean: string, concavity_mean: string, concave points_mean: string, symmetry_mean: string, fractal_dimension_mean: string, radius_se: string, texture_se: string, perimeter_se: string, area_se: string, smoothness_se: string, compactness_se: string, concavity_se: string, concave points_se: string, symmetry_se: string, fractal_dimension_se: string, radius_worst: string, texture_worst: string, perimeter_worst: string, area_worst: string, smoothness_worst: string, compactness_worst: string, concavity_worst: string, concave points_worst: string, symmetry_worst: string, fractal_dimension_worst: string, _c32: string]

In [6]:
df_spark.show(5)

+--------+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+----+
|      id|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|_c32|
+--------+---------+-----------+------

In [9]:
import pandas as pd

In [13]:
df_spark_mod = df_spark.drop('id','_c32')
df_spark_mod.show(2)

+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+----------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+-----------------------+
|diagnosis|radius_mean|texture_mean|perimeter_mean|area_mean|smoothness_mean|compactness_mean|concavity_mean|concave points_mean|symmetry_mean|fractal_dimension_mean|radius_se|texture_se|perimeter_se|area_se|smoothness_se|compactness_se|concavity_se|concave points_se|symmetry_se|fractal_dimension_se|radius_worst|texture_worst|perimeter_worst|area_worst|smoothness_worst|compactness_worst|concavity_worst|concave points_worst|symmetry_worst|fractal_dimension_worst|
+---------+-----------+------------+--------------+---------+-----

In [14]:
df_spark_mod = df_spark.select('diagnosis','radius_mean','perimeter_mean','area_mean')
df_spark_mod.show(2)

+---------+-----------+--------------+---------+
|diagnosis|radius_mean|perimeter_mean|area_mean|
+---------+-----------+--------------+---------+
|        M|      17.99|         122.8|     1001|
|        M|      20.57|         132.9|     1326|
+---------+-----------+--------------+---------+
only showing top 2 rows



In [15]:
df_spark_mod2 = df_spark_mod.withColumnRenamed("perimeter_mean", "perimeter_avg")
df_spark_mod2.show(2)

+---------+-----------+-------------+---------+
|diagnosis|radius_mean|perimeter_avg|area_mean|
+---------+-----------+-------------+---------+
|        M|      17.99|        122.8|     1001|
|        M|      20.57|        132.9|     1326|
+---------+-----------+-------------+---------+
only showing top 2 rows



In [16]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col, lit, when

In [17]:
df_spark_mod = df_spark_mod.withColumn('base',lit("Cancer_Data"))
df_spark_mod.show(2)

+---------+-----------+--------------+---------+-----------+
|diagnosis|radius_mean|perimeter_mean|area_mean|       base|
+---------+-----------+--------------+---------+-----------+
|        M|      17.99|         122.8|     1001|Cancer_Data|
|        M|      20.57|         132.9|     1326|Cancer_Data|
+---------+-----------+--------------+---------+-----------+
only showing top 2 rows



In [18]:
df_spark_mod = df_spark_mod.withColumn('area_aprox',3.1415*col('radius_mean')**2)
df_spark_mod.show(2)

+---------+-----------+--------------+---------+-----------+------------------+
|diagnosis|radius_mean|perimeter_mean|area_mean|       base|        area_aprox|
+---------+-----------+--------------+---------+-----------+------------------+
|        M|      17.99|         122.8|     1001|Cancer_Data|1016.7153741499999|
|        M|      20.57|         132.9|     1326|Cancer_Data|1329.2468733500002|
+---------+-----------+--------------+---------+-----------+------------------+
only showing top 2 rows



In [19]:
df_spark_mod = df_spark_mod.withColumn('area_aprox',F.round(col('area_aprox')))
df_spark_mod.show(2)

+---------+-----------+--------------+---------+-----------+----------+
|diagnosis|radius_mean|perimeter_mean|area_mean|       base|area_aprox|
+---------+-----------+--------------+---------+-----------+----------+
|        M|      17.99|         122.8|     1001|Cancer_Data|    1017.0|
|        M|      20.57|         132.9|     1326|Cancer_Data|    1329.0|
+---------+-----------+--------------+---------+-----------+----------+
only showing top 2 rows



In [20]:
df_spark_mod = df_spark_mod.withColumn('diagnosis_desc',
                                        when(col('diagnosis')=='M','malignant')
                                       .when(col('diagnosis')=='B','benign')
                                       .otherwise('others'))
df_spark_mod.show(2)

+---------+-----------+--------------+---------+-----------+----------+--------------+
|diagnosis|radius_mean|perimeter_mean|area_mean|       base|area_aprox|diagnosis_desc|
+---------+-----------+--------------+---------+-----------+----------+--------------+
|        M|      17.99|         122.8|     1001|Cancer_Data|    1017.0|     malignant|
|        M|      20.57|         132.9|     1326|Cancer_Data|    1329.0|     malignant|
+---------+-----------+--------------+---------+-----------+----------+--------------+
only showing top 2 rows



In [21]:
df_benign = df_spark_mod.filter(col('diagnosis_desc')=='benign')
df_benign.show(3)

+---------+-----------+--------------+---------+-----------+----------+--------------+
|diagnosis|radius_mean|perimeter_mean|area_mean|       base|area_aprox|diagnosis_desc|
+---------+-----------+--------------+---------+-----------+----------+--------------+
|        B|      13.54|         87.46|    566.3|Cancer_Data|     576.0|        benign|
|        B|      13.08|         85.63|      520|Cancer_Data|     537.0|        benign|
|        B|      9.504|         60.34|    273.9|Cancer_Data|     284.0|        benign|
+---------+-----------+--------------+---------+-----------+----------+--------------+
only showing top 3 rows



In [22]:
df_benign = df_spark_mod.filter("diagnosis_desc =='benign'")
df_benign.show(3)

+---------+-----------+--------------+---------+-----------+----------+--------------+
|diagnosis|radius_mean|perimeter_mean|area_mean|       base|area_aprox|diagnosis_desc|
+---------+-----------+--------------+---------+-----------+----------+--------------+
|        B|      13.54|         87.46|    566.3|Cancer_Data|     576.0|        benign|
|        B|      13.08|         85.63|      520|Cancer_Data|     537.0|        benign|
|        B|      9.504|         60.34|    273.9|Cancer_Data|     284.0|        benign|
+---------+-----------+--------------+---------+-----------+----------+--------------+
only showing top 3 rows



In [23]:
df_benign2 = df_spark_mod.filter((col('diagnosis_desc')=='benign')&
                                 (col('radius_mean')<=10))
df_benign2.show(3)

+---------+-----------+--------------+---------+-----------+----------+--------------+
|diagnosis|radius_mean|perimeter_mean|area_mean|       base|area_aprox|diagnosis_desc|
+---------+-----------+--------------+---------+-----------+----------+--------------+
|        B|      9.504|         60.34|    273.9|Cancer_Data|     284.0|        benign|
|        B|      8.196|         51.71|    201.9|Cancer_Data|     211.0|        benign|
|        B|      8.618|         54.34|    224.5|Cancer_Data|     233.0|        benign|
+---------+-----------+--------------+---------+-----------+----------+--------------+
only showing top 3 rows



In [24]:
print("Base full:   ",df_spark_mod.count())
print("Base Benign: ",df_benign.count())
print("Base Benign2:",df_benign2.count())

Base full:    569
Base Benign:  357
Base Benign2: 84
